In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.tsa.api import ARDL
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df = pd.read_excel('../comparing_ir.xlsx',usecols=[0,1])
df.index = pd.date_range('2010-06','2022-04',freq='m')

df

,TUR_Policy_Rate,FED_Policy_Rate
2010-06-30,7.0,0.25
2010-07-31,7.0,0.25
2010-08-31,7.0,0.25
2010-09-30,7.0,0.25
2010-10-31,7.0,0.25
...,...,...
2021-11-30,15.0,0.25
2021-12-31,14.0,0.25
2022-01-31,14.0,0.25
2022-02-28,14.0,0.25


In [3]:
before_covid = df[(df.index <'2020-03')]
after_covid = df.iloc[112:142,:] 
print(f'n of post corona : {before_covid.shape}\nn of after corona : {after_covid.shape}')

n of post corona : (117, 2)
n of after corona : (30, 2)


In [4]:
model_post_corona,model_corona = sm.OLS(before_covid['TUR_Policy_Rate'],sm.add_constant(before_covid['FED_Policy_Rate'])),sm.OLS(after_covid['TUR_Policy_Rate'],sm.add_constant(after_covid['FED_Policy_Rate']))
results_post_corona,results_corona = model_post_corona.fit(),model_corona.fit()

results_post_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TUR_Policy_Rate   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     355.5
Date:                Tue, 17 May 2022   Prob (F-statistic):           5.60e-37
Time:                        15:20:59   Log-Likelihood:                -280.07
No. Observations:                 117   AIC:                             564.1
Df Residuals:                     115   BIC:                             569.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               4.5359      0.353     12.857      0.000       3.837       5.235
FED_Policy_Rate     6.1359      0.325     18.855      0.000       5.491       6.781
==============================================================================
Omnibus:                        6.524   Durbin-Watson:                   0.189
Prob(Omnibus):                  0.038   Jarque-Bera (JB):                6.134
Skew:                          -0.463   Prob(JB):                       0.0466
Kurtosis:                       3.632   Cond. No.                         2.46
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
results_corona.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TUR_Policy_Rate   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.006
Date:                Tue, 17 May 2022   Prob (F-statistic):              0.325
Time:                        15:21:11   Log-Likelihood:                -81.876
No. Observations:                  30   AIC:                             167.8
Df Residuals:                      28   BIC:                             170.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              14.6077      0.948     15.402      0.000      12.665      16.550
FED_Policy_Rate    -1.2611      1.258     -1.003      0.325      -3.837       1.315
==============================================================================
Omnibus:                        5.021   Durbin-Watson:                   0.136
Prob(Omnibus):                  0.081   Jarque-Bera (JB):                2.223
Skew:                          -0.350   Prob(JB):                        0.329
Kurtosis:                       1.864   Cond. No.                         2.40
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

There is no significant affect on TCMB Policy Rates

Focusing Post Corona period

In [6]:
print(f'Breusch-Pagan on Post Corona P Value : {sms.het_breuschpagan(results_post_corona.resid,results_post_corona.model.exog)[1]}')

Breusch-Pagan on Post Corona P Value : 1.1112922387704073e-10


In [7]:
##Seems its Heteroscedastic

In [13]:
print(f'Breusch-Godfrey P Value for Serial Autocorrelation : {sms.acorr_breusch_godfrey(results_post_corona)[1]}')

Breusch-Godfrey P Value for Serial Autocorrelation : 5.753759613723248e-17


In [11]:
print(f'Durbin-Watson Test Statistic is {sms.durbin_watson(results_post_corona.resid)}')

Durbin-Watson Test Statistic is 0.1888966184409973


In [12]:
##Seems has Positive Autocorrelation 

In [15]:
ardl = ARDL(before_covid['TUR_Policy_Rate'],[1,2],sm.add_constant(before_covid['FED_Policy_Rate']),{'FED_Policy_Rate':2})
results_post_corona = ardl.fit()

results_post_corona.summary()

c:\Users\acer\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\ardl\model.py:158: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: const.
  warnings.warn(msg, SpecificationWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARDL Model Results                              
==============================================================================
Dep. Variable:        TUR_Policy_Rate   No. Observations:                  117
Model:                     ARDL(2, 2)   Log Likelihood                -176.584
Method:               Conditional MLE   S.D. of innovations              1.124
Date:                Tue, 17 May 2022   AIC                            367.167
Time:                        15:26:20   BIC                            386.382
Sample:                    08-31-2010   HQIC                           374.966
                         - 02-29-2020                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.3011      0.255      1.182      0.240      -0.204       0.806
TUR_Policy_Rate.L1     0.9820      0.096     10.252      0.000       0.792       1.172
TUR_Policy_Rate.L2    -0.0612      0.099     -0.621      0.536      -0.256       0.134
FED_Policy_Rate.L0     8.2987      1.383      6.001      0.000       5.558      11.039
FED_Policy_Rate.L1    -8.3346      2.118     -3.935      0.000     -12.532      -4.137
FED_Policy_Rate.L2     0.5084      1.641      0.310      0.757      -2.744       3.761
======================================================================================
"""

In [16]:
##Checking Diagnostics

results_post_corona.test_normality()

Jarque-Bera    1959.354532
P-value           0.000000
Skewness          3.257415
Kurtosis         22.143295
dtype: float64

In [22]:
results_post_corona.test_heteroskedasticity(1)

,ARCH-LM,P-value,DF
Lag,,,
1,0.181885,0.669759,1


In [19]:
endog = before_covid['TUR_Policy_Rate'][2:]
exog = sm.add_constant(before_covid['FED_Policy_Rate'])[2:]

print(f'Durbin Watson Test Statistics : {sms.durbin_watson(results_post_corona.resid)}')

Durbin Watson Test Statistics : 2.0062928688659913


In [20]:
from sklearn.metrics import r2_score

print(f'Post Corona Period R² : {r2_score(endog,results_post_corona.fittedvalues)}')

Post Corona Period R² : 0.9566935869061012
